In [1]:
import os
import numpy as np
import pandas as pd
import pickle

In [2]:
MODELS_DIR='models'
# load model
model_file = open(os.path.join(MODELS_DIR, 'dataset_reduced.csv_HOG__power_19-01-2023 18-42-58_model.sav'), 'rb')
model = pickle.load(model_file)
print(model.score)



0.8325563872692063


In [3]:
print(model.cv_results)

{'fit_time': array([0.24032664, 0.22512603, 0.22271538, 0.23613524]), 'score_time': array([0.0044899 , 0.0054245 , 0.00472212, 0.00483465]), 'test_score': array([0.85876349, 0.7881519 , 0.83568457, 0.84762558]), 'train_score': array([0.86815357, 0.88444292, 0.87377594, 0.87021472])}
